In [1]:
%env PYTORCH_ENABLE_MPS_FALLBACK

'1'

In [2]:
from fasttext_classifier.model import FastTextClassifier, FastTextClassifierConfig
from fasttext_classifier.encoder import FastTextEncoder
import torch
from datasets import load_dataset
from torch.utils.data import DataLoader
from pytorch_lightning import loggers as pl_loggers
from pytorch_lightning import Trainer

In [3]:
dataset_name = "ag_news"
dataset = load_dataset(dataset_name)

Found cached dataset ag_news (/Users/joseph.lee/.cache/huggingface/datasets/ag_news/default/0.0.0/bc2bcb40336ace1a0374767fc29bb0296cdaf8a6da7298436239c54d79180548)


  0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 120000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 7600
    })
})

In [5]:
config = FastTextClassifierConfig(
    num_classes=4,
    batch_size=256,
    lr=0.5,
    min_n=2,
    max_n=6,
    word_ngrams=2,
    dim=10,
    bucket=10000,
)

initialize tokenizer

In [6]:
def _tokenize(s):
    return s.split()

def collate_batch(batch):
    label_list = torch.LongTensor([x["label"] for x in batch])
    out = tokenizer(
        [_tokenize(x["text"]) for x in batch], return_tensors="pt", ft_mode=True
    )
    # input_ids = out["input_ids"]
    return {
        "label": label_list,
        "input_ids": out["input_ids"]
    }

In [7]:
tokenizer = FastTextEncoder([_tokenize(x) for x in dataset["train"]["text"]], config=config)

In [8]:
config.vocab_size = tokenizer.vocab_size

In [9]:
trainloader = DataLoader(
    dataset["train"], batch_size=config.batch_size, shuffle=False, collate_fn=collate_batch
)
testloader = DataLoader(
    dataset["test"], batch_size=config.batch_size, shuffle=False, collate_fn=collate_batch
)

In [10]:
model = FastTextClassifier(config)

In [11]:
trainer = Trainer(
    logger=[pl_loggers.TensorBoardLogger(save_dir="./"), pl_loggers.CSVLogger(save_dir="./")],
    accelerator="cpu",
    max_steps=100,
    # max_epochs=2,
)

GPU available: True (mps), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/opt/homebrew/Caskroom/miniforge/base/envs/ml_fasttext/lib/python3.9/site-packages/pytorch_lightning/trainer/setup.py:200: UserWarning: MPS available but not used. Set `accelerator` and `devices` using `Trainer(accelerator='mps', devices=1)`.
  rank_zero_warn(


In [12]:
trainer.fit(model, trainloader, testloader)


  | Name       | Type                | Params
---------------------------------------------------
0 | criterion  | CrossEntropyLoss    | 0     
1 | embedding  | Embedding           | 2.0 M 
2 | fc1        | Linear              | 44    
3 | val_acc    | MulticlassAccuracy  | 0     
4 | val_prec   | MulticlassPrecision | 0     
5 | val_recall | MulticlassRecall    | 0     
6 | val_f1     | MulticlassF1Score   | 0     
---------------------------------------------------
2.0 M     Trainable params
0         Non-trainable params
2.0 M     Total params
7.925     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

/opt/homebrew/Caskroom/miniforge/base/envs/ml_fasttext/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:224: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/opt/homebrew/Caskroom/miniforge/base/envs/ml_fasttext/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:224: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=2` reached.


In [13]:
trainer.validate(model, testloader)

Validation: 0it [00:00, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃      Validate metric      ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         eval:acc          │    0.9040789604187012     │
│       eval:f1score        │    0.9034754037857056     │
│      eval:precision       │    0.9034491181373596     │
│        eval:recall        │    0.9040789008140564     │
└───────────────────────────┴───────────────────────────┘

[{'eval:acc': 0.9040789604187012,
  'eval:precision': 0.9034491181373596,
  'eval:recall': 0.9040789008140564,
  'eval:f1score': 0.9034754037857056}]

In [14]:
outs = trainer.predict(model, testloader)

/opt/homebrew/Caskroom/miniforge/base/envs/ml_fasttext/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:224: PossibleUserWarning: The dataloader, predict_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Predicting: 469it [00:00, ?it/s]

In [18]:
outs[0]

tensor([[ 1.2413, -1.0855,  1.3304, -1.0577],
        [-1.8478, -6.4690, -2.9572,  8.4749],
        [-2.2921, -3.7996, -0.4137,  4.5982],
        ...,
        [ 2.4083, -5.1486,  5.5299, -3.0221],
        [ 1.0118, -3.5451,  4.1630, -1.9272],
        [ 0.0621, -6.4624, -1.0199,  5.3991]])